In [ ]:
# The URL of the MISP instance to connect to
misp_url = 'https://127.0.0.1:8443'
# Can be found in the MISP web interface under ||
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'd6OmdDFvU3Seau3UjwvHS1y3tFQbaRNhJhDX0tjh'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

# Getting the API key (automatically generated on the trainig VM)

In [ ]:
from pathlib import Path

api_file = Path('apikey')
if api_file.exists():
    misp_url = 'http://127.0.0.1'
    misp_verifycert = False
    with open(api_file) as f:
        misp_key = f.read().strip()
    print(misp_key)

# Initialize PyMISP - NG

In [ ]:
from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert, debug=False)

# Index Search (fast, only returns events metadata)

## Search unpublished events

**WARNING**: By default, the search query will only return all the events listed on the index page

In [ ]:
r = misp.search(published=False, metadata=True)
print(r)

## Get the meta data of events

In [ ]:
r = misp.search(eventid=[1,2,3], metadata=True, pythonify=True)

In [ ]:
r

## Search Tag & mix with other parameters

In [ ]:
r = misp.search(tags=['tlp:white'], metadata=True, pythonify=True)
for e in r:
    print(e)

In [ ]:
print('No attributes are in the event', r[0].attributes)

In [ ]:
r = misp.search(tags='TODO:VT-ENRICHMENT', published=False)

In [ ]:
r = misp.search(tags=['!TODO:VT-ENRICHMENT', 'tlp:white'], metadata=True, published=False)  # ! means "not this tag"

## Full text search on event info field

In [ ]:
r = misp.search(eventinfo='circl', metadata=True)

## Search by org

In [ ]:
r = misp.search(org='CIRCL', metadata=True)

## Search updated events

In [ ]:
r = misp.search(timestamp='1h', metadata=True)

# Search full events (Slower, returns full events)

## Getting timestamps

In [ ]:
from datetime import datetime, date, timedelta
from dateutil.parser import parse

int(datetime.now().timestamp())

d = parse('2018-03-24')
int(d.timestamp())

today = int(datetime.today().timestamp())
yesterday = int((datetime.today() - timedelta(days=1)).timestamp())

print(today, yesterday)



In [ ]:
complex_query = misp.build_complex_query(or_parameters=['uibo.lembit@mail.ee', '103.195.185.222'])

In [ ]:
print(complex_query)

In [ ]:
complex_query = misp.build_complex_query(or_parameters=['59.157.4.2', 'hotfixmsupload.com', '8.8.8.8'])
events = misp.search(value=complex_query, pythonify=True)

for e in events:
    print(e)

In [ ]:
r = misp.search(value=complex_query, pythonify=True)
print(r)

In [ ]:
r = misp.search(category='Payload delivery')

In [ ]:
r = misp.search(value='uibo.lembit@mail.ee', metadata=True, pythonify=True) # no attributes

In [ ]:
r = misp.search(timestamp=['2h', '1h'])

In [ ]:
r = misp.search(value='8.8.8.8', enforceWarninglist=True)

In [ ]:
r = misp.search(value='8.8.8.8', deleted=True)

In [ ]:
r = misp.search(value='8.8.8.8', publish_timestamp=1521846000)  # everything published since that timestamp

In [ ]:
r = misp.search(value='8.8.8.8', last='1d')  # everything published in the last <interval>

In [ ]:
r = misp.search(value='8.8.8.8', timestamp=[yesterday, today])  # everything updated since that timestamp

In [ ]:
r = misp.search(value='8.8.8.8', withAttachments=True)  # Return attachments

In [ ]:
r = misp.search(tags=['%tlp:amber%'], pythonify=True)

In [ ]:
print(r[0].tags)

# Search for attributes

In [ ]:
r = misp.search(controller='attributes', value='8.8.8.8')

In [ ]:
r = misp.search(controller='attributes', value='wrapper.no', event_timestamp='5d')  # only consider events updated since this timestamp

In [ ]:
print(r)

In [ ]:
# Search attributes (specified in controller) where the attribute type is 'ip-src'
# And the to_ids flag is set
attributes = misp.search(controller='attributes', type_attribute='ip-src', to_ids=0, pythonify=True)

event_ids = set()
for attr in attributes:
    event_ids.add(attr.event_id)

# Fetch all related events
for event_id in event_ids:
    event = misp.get_event(event_id)
    print(event.info)

## Last *published* attributes

In [ ]:
attributes = misp.search(controller='attributes', publish_timestamp='1d', pythonify=True)

for attribute in attributes:
    print(attribute.event_id, attribute)

In [ ]:
attributes = misp.search(controller='attributes', publish_timestamp=['2d', '1h'], pythonify=True)

for a in attributes:
    print(a)

## Last *updated* attributes

In [ ]:
from datetime import datetime

ts = int(datetime.now().timestamp())

attributes = misp.search(controller='attributes', timestamp=ts - 36000, pythonify=True)

for a in attributes:
    print(a)

## Orther output formats

**Warning**: For that to work, the matching event has to be published

In [ ]:
r = misp.search(controller='attributes', value='8.8.8.8', return_format='csv')

In [ ]:
r = misp.search(controller='events', value='9.8.8.8', return_format='snort')

In [ ]:
r = misp.search(controller='events', value='9.8.8.8', return_format='suricata')

In [ ]:
r = misp.search(controller='events', value='9.8.8.8', return_format='stix')

In [ ]:
r = misp.search(controller='events', value='9.8.8.8', return_format='stix2')

In [ ]:
print(r)

## Search in logs

In [ ]:
logs = misp.search_logs(model='Tag', title='tlp:white')
print(logs)

In [ ]:
logs = misp.search_logs(model='Event', pythonify=True)
#print(logs)
for l in logs:
    print(l.title)